In [1]:
%run functions.ipynb
%load_ext autoreload
%autoreload 2

Populating the interactive namespace from numpy and matplotlib


In [2]:
X_train, X_test, y_train, y_test = load_train_and_test_parts()
print("Train and test sizes: {} {}".format(X_train.shape, X_test.shape))
print(
    "(1, 0) labels count in train test: {} {}".format(
        (np.count_nonzero(y_train == 1), np.count_nonzero(y_train == 0)),
        (np.count_nonzero(y_test == 1), np.count_nonzero(y_test == 0)),
    )
)

Train and test sizes: (726, 12179) (243, 12179)
(1, 0) labels count in train test: (289, 437) (104, 139)


In [3]:
scoring = 'roc_auc'
cv = 4
cv_out = 5
cv_in = 2
n_iter = 1
n_jobs = 3

In [4]:
# Parameters
scoring = "roc_auc"
cv = 4
cv_out = 5
cv_in = 2
n_iter = 50
n_jobs = 4


In [5]:
rlr_params = {'C' : np.linspace(0.1, 5, 50), 'n_estimators' : [100, 250, 500, 1000], 'n_variables' : [100, 250, 500, 1000]}
rlr_cv = RandomizedSearchCV(RandomLogisticsRegressions(penalty='l1'), cv=cv, scoring=scoring, n_iter=n_iter, n_jobs=n_jobs, 
                                iid=False, param_distributions=rlr_params, random_state=random_state)

clf_rlr = fit_clf_print_scores(rlr_cv, X_train, y_train, X_test, y_test)

/pio/scratch/1/i272955/anaconda3/envs/master/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


/pio/scratch/1/i272955/anaconda3/envs/master/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


/pio/scratch/1/i272955/anaconda3/envs/master/lib/python3.7/site-packages/sklearn/externals/joblib/externals/loky/process_executor.py:706: UserWarning: A worker stopped while some jobs were given to the executor. This can be caused by a too short worker timeout or by a memory leak.
  "timeout or by a memory leak.", UserWarning


{'n_variables': 250, 'n_estimators': 100, 'C': 3.4000000000000004}


,ROC AUC,Precision,Recall,F1
train,0.985,0.974,0.903,0.937
test,0.815,0.717,0.635,0.673


In [6]:
all_scores_names = ['roc_auc', 'precision', 'recall', 'f1']
estimated_scores = {}
randomized_cvs = {}
models = {
    'Random logistic regression': {
        'model': RandomLogisticsRegressions(penalty='l1'),
        'params': {'C' : np.linspace(0.1, 5, 50), 'n_estimators' : [100, 250, 500, 1000], 'n_variables' : [100, 250, 500, 1000]}
    }
}

In [7]:
for model_name, model_specification in models.items():
    rcv = RandomizedSearchCV(estimator=model_specification['model'], param_distributions=model_specification['params'],
                             cv=cv_in, scoring=scoring, n_iter=n_iter, iid=False, random_state=random_state)
    randomized_cvs[model_name] = rcv

In [ ]:
for name, rcv in randomized_cvs.items():
    nested_scores = cross_validate(rcv, X_train, y_train, scoring = all_scores_names, cv=cv_out)
    estimated_scores[name] = {}
    for score_name in all_scores_names:
        estimated_scores[name][score_name] = {
            'mean': np.mean(nested_scores['test_' + score_name]),
            'std': np.std(nested_scores['test_' + score_name])
        }
    print('Done for ' + name)

In [ ]:
print_summarized_scores(estimated_scores, models, latex=True)